In [ ]:
import sys, os
import time
import chess
import chess.svg
from IPython.display import display, SVG
sys.path.append(os.path.abspath(".."))

from core.gaviota import get_move_from_table
from core.evaluation import evaluate_board
from core.transposition_table import ZobristBoard, LRUCache, Entry

transposition_table = LRUCache(1000000)

In [ ]:
ADVANTAGE_THRESHOLD = 150      
REPETITION_PENALTY   = 50000

def is_repetition_n_or_more(position, n=2):
    try:
        stack = getattr(position, "hash_stack", None)
        key = getattr(position, "zobrist_hash", None)
        if stack is not None and key is not None:
            return stack.count(key) >= n
    except Exception:
        pass
    try:
        return position.is_repetition()
    except Exception:
        return False

def get_victim_type(position, move):
    if position.is_en_passant(move):
        return chess.PAWN
    piece = position.piece_at(move.to_square)
    return piece.piece_type if piece else None

def order_moves(position, moves, tt_best_move=None):
    def move_score(move):
        score = 0
        if tt_best_move and move == tt_best_move:
            score += 10000
        victim_type = get_victim_type(position, move)
        if victim_type:
            attacker = position.piece_at(move.from_square)
            attacker_type = attacker.piece_type if attacker else 0
            score += 10 * victim_type - attacker_type
        if move.promotion:
            score += 5 * (move.promotion or 0)
        return score
    return sorted(moves, key=move_score, reverse=True)

def quiescence(position, alpha, beta, maximizingPlayer, qs_depth=0, max_qs_depth=4):
    if qs_depth > max_qs_depth or position.is_game_over():
        return evaluate_board(position)

    stand_pat = evaluate_board(position)
    if maximizingPlayer:
        if stand_pat >= beta:
            return beta
        alpha = max(alpha, stand_pat)
    else:
        if stand_pat <= alpha:
            return alpha
        beta = min(beta, stand_pat)

    # captures only (and promotions)
    captures = []
    for m in position.legal_moves:
        if position.is_capture(m) or m.promotion:
            captures.append(m)
    if not captures:
        return stand_pat

    captures = order_moves(position, captures)

    if maximizingPlayer:
        value = float('-inf')
        for move in captures:
            position.push(move)
            if is_repetition_n_or_more(position, 2):
                score = 0
            else:
                score = quiescence(position, alpha, beta, False, qs_depth + 1, max_qs_depth)
            position.pop()

            if score >= beta:
                return beta
            alpha = max(alpha, score)
            value = max(value, score)
        return value if value != float('-inf') else alpha
    else:
        value = float('inf')
        for move in captures:
            position.push(move)
            if is_repetition_n_or_more(position, 2):
                score = 0
            else:
                score = quiescence(position, alpha, beta, True, qs_depth + 1, max_qs_depth)
            position.pop()

            if score <= alpha:
                return alpha
            beta = min(beta, score)
            value = min(value, score)
        return value if value != float('inf') else beta

def pvs(position, depth, alpha, beta, maximizingPlayer):
    state_key = getattr(position, "zobrist_hash", None)

    tt_best_move = None
    if state_key is not None and state_key in transposition_table:
        entry = transposition_table[state_key]
        if entry.depth >= depth:
            if entry.flag == 'exact':
                return entry.value, entry.best_move
            elif entry.flag == 'lowerbound' and entry.value >= beta:
                return entry.value, entry.best_move
            elif entry.flag == 'upperbound' and entry.value <= alpha:
                return entry.value, entry.best_move
        tt_best_move = entry.best_move

    if depth == 0 or position.is_game_over():
        val = evaluate_board(position)
        if state_key is not None:
            transposition_table[state_key] = Entry(val, depth, 'exact', None)
        return val, None

    alpha_orig = alpha
    beta_orig = beta
    best_move = None

    moves = order_moves(position, list(position.legal_moves), tt_best_move)
    if not moves:
        val = evaluate_board(position)
        if state_key is not None:
            transposition_table[state_key] = Entry(val, depth, 'exact', None)
        return val, None

    if maximizingPlayer:
        value = float('-inf')
        first = True
        for move in moves:
            position.push(move)
            if is_repetition_n_or_more(position, 2):
                score = 0
            else:
                if first:
                    score, _ = pvs(position, depth-1, alpha, beta, False)
                else:
                    score, _ = pvs(position, depth-1, alpha, alpha+1, False)
                    if score > alpha and score < beta:
                        score, _ = pvs(position, depth-1, score, beta, False)
            position.pop()

            if score > value:
                value = score
                best_move = move
            alpha = max(alpha, score)
            if alpha >= beta:
                break
            first = False
    else:
        value = float('inf')
        first = True
        for move in moves:
            position.push(move)
            if is_repetition_n_or_more(position, 2):
                score = 0
            else:
                if first:
                    score, _ = pvs(position, depth-1, alpha, beta, True)
                else:
                    score, _ = pvs(position, depth-1, beta-1, beta, True)
                    if score > alpha and score < beta:
                        score, _ = pvs(position, depth-1, alpha, score, True)
            position.pop()

            if score < value:
                value = score
                best_move = move
            beta = min(beta, score)
            if alpha >= beta:
                break
            first = False

    flag = 'exact'
    if value <= alpha_orig:
        flag = 'upperbound'
    elif value >= beta_orig:
        flag = 'lowerbound'
    if state_key is not None:
        transposition_table[state_key] = Entry(value, depth, flag, best_move)

    return value, best_move

def iterative_deepening(position, max_depth, maximizingPlayer=True, time_limit=None, use_pvs=True):
    start_time = time.time()
    best_move = None
    best_value = None
    reached = 0

    for d in range(1, max_depth + 1):
        if time_limit is not None and (time.time() - start_time) >= time_limit:
            break
        if use_pvs:
            val, mv = pvs(position, d, float('-inf'), float('inf'), maximizingPlayer)
        else:
            val, mv = minimax(position, d, float('-inf'), float('inf'), maximizingPlayer)
        if mv is not None:
            best_move = mv
            best_value = val
        reached = d
        if best_value is not None and abs(best_value) >= 90000:
            break

    elapsed = time.time() - start_time
    return best_value, best_move, reached, elapsed

def iterative_deepening_with_qs(position, max_depth, maximizingPlayer=True, time_limit=None, use_pvs=True):
    start_time = time.time()
    best_move = None
    best_value = None
    reached = 0

    for d in range(1, max_depth + 1):
        if time_limit is not None and (time.time() - start_time) >= time_limit:
            break
        if use_pvs:
            val, mv = pvs_with_qs(position, d, float('-inf'), float('inf'), maximizingPlayer)
        else:
            val, mv = minimax_with_qs(position, d, float('-inf'), float('inf'), maximizingPlayer)
        if mv is not None:
            best_move = mv
            best_value = val
        reached = d
        if best_value is not None and abs(best_value) >= 90000:
            break

    elapsed = time.time() - start_time
    return best_value, best_move, reached, elapsed

def pvs_with_qs(position, depth, alpha, beta, maximizingPlayer):
    state_key = getattr(position, "zobrist_hash", None)

    tt_best_move = None
    if state_key is not None and state_key in transposition_table:
        entry = transposition_table[state_key]
        if entry.depth >= depth:
            if entry.flag == 'exact':
                return entry.value, entry.best_move
            elif entry.flag == 'lowerbound' and entry.value >= beta:
                return entry.value, entry.best_move
            elif entry.flag == 'upperbound' and entry.value <= alpha:
                return entry.value, entry.best_move
        tt_best_move = entry.best_move

    if depth == 0 or position.is_game_over():
        val = quiescence(position, alpha, beta, maximizingPlayer)
        if state_key is not None:
            transposition_table[state_key] = Entry(val, depth, 'exact', None)
        return val, None

    alpha_orig = alpha
    beta_orig = beta
    best_move = None

    moves = order_moves(position, list(position.legal_moves), tt_best_move)
    if not moves:
        val = quiescence(position, alpha, beta, maximizingPlayer)
        if state_key is not None:
            transposition_table[state_key] = Entry(val, depth, 'exact', None)
        return val, None

    if maximizingPlayer:
        value = float('-inf')
        first = True
        for move in moves:
            position.push(move)
            if is_repetition_n_or_more(position, 2):
                score = 0
            else:
                if first:
                    score, _ = pvs_with_qs(position, depth-1, alpha, beta, False)
                else:
                    score, _ = pvs_with_qs(position, depth-1, alpha, alpha+1, False)
                    if score > alpha and score < beta:
                        score, _ = pvs_with_qs(position, depth-1, score, beta, False)
            position.pop()

            if score > value:
                value = score
                best_move = move
            alpha = max(alpha, score)
            if alpha >= beta:
                break
            first = False
    else:
        value = float('inf')
        first = True
        for move in moves:
            position.push(move)
            if is_repetition_n_or_more(position, 2):
                score = 0
            else:
                if first:
                    score, _ = pvs_with_qs(position, depth-1, alpha, beta, True)
                else:
                    score, _ = pvs_with_qs(position, depth-1, beta-1, beta, True)
                    if score > alpha and score < beta:
                        score, _ = pvs_with_qs(position, depth-1, alpha, score, True)
            position.pop()

            if score < value:
                value = score
                best_move = move
            beta = min(beta, score)
            if alpha >= beta:
                break
            first = False

    flag = 'exact'
    if value <= alpha_orig:
        flag = 'upperbound'
    elif value >= beta_orig:
        flag = 'lowerbound'
    if state_key is not None:
        transposition_table[state_key] = Entry(value, depth, flag, best_move)
    return value, best_move

def minimax_with_qs(position, depth, alpha, beta, maximizingPlayer):
    state_key = getattr(position, "zobrist_hash", None)
    tt_best_move = None
    if state_key is not None and state_key in transposition_table:
        entry = transposition_table[state_key]
        if entry.depth >= depth:
            if entry.flag == 'exact':
                return entry.value, entry.best_move
            elif entry.flag == 'lowerbound' and entry.value >= beta:
                return entry.value, entry.best_move
            elif entry.flag == 'upperbound' and entry.value <= alpha:
                return entry.value, entry.best_move
        tt_best_move = entry.best_move

    if depth == 0 or position.is_game_over():
        val = quiescence(position, alpha, beta, maximizingPlayer)
        if state_key is not None:
            transposition_table[state_key] = Entry(val, depth, 'exact', None)
        return val, None

    alpha_orig = alpha
    beta_orig = beta
    best_move = None

    moves = order_moves(position, list(position.legal_moves), tt_best_move)

    if maximizingPlayer:
        maxEval = float('-inf')
        for move in moves:
            position.push(move)
            if is_repetition_n_or_more(position, 2):
                evaluation = 0
            else:
                evaluation, _ = minimax_with_qs(position, depth-1, alpha, beta, False)
            position.pop()
            if evaluation > maxEval:
                maxEval = evaluation
                best_move = move
            alpha = max(alpha, evaluation)
            if beta <= alpha:
                break
        value = maxEval
    else:
        minEval = float('inf')
        for move in moves:
            position.push(move)
            if is_repetition_n_or_more(position, 2):
                evaluation = 0
            else:
                evaluation, _ = minimax_with_qs(position, depth-1, alpha, beta, True)
            position.pop()
            if evaluation < minEval:
                minEval = evaluation
                best_move = move
            beta = min(beta, evaluation)
            if beta <= alpha:
                break
        value = minEval

    flag = 'exact'
    if value <= alpha_orig:
        flag = 'upperbound'
    elif value >= beta_orig:
        flag = 'lowerbound'
    if state_key is not None:
        transposition_table[state_key] = Entry(value, depth, flag, best_move)

    return value, best_move

def minimax(position, depth, alpha, beta, maximizingPlayer):
    state_key = getattr(position, "zobrist_hash", None)
    tt_best_move = None
    if state_key is not None and state_key in transposition_table:
        entry = transposition_table[state_key]
        if entry.depth >= depth:
            if entry.flag == 'exact':
                return entry.value, entry.best_move
            elif entry.flag == 'lowerbound' and entry.value >= beta:
                return entry.value, entry.best_move
            elif entry.flag == 'upperbound' and entry.value <= alpha:
                return entry.value, entry.best_move
        tt_best_move = entry.best_move

    if depth == 0 or position.is_game_over():
        value = evaluate_board(position)
        if state_key is not None:
            transposition_table[state_key] = Entry(value, depth, 'exact', None)
        return value, None

    alpha_orig = alpha
    beta_orig = beta
    best_move = None

    moves = order_moves(position, list(position.legal_moves), tt_best_move)

    if maximizingPlayer:
        maxEval = float('-inf')
        for move in moves:
            position.push(move)
            if is_repetition_n_or_more(position, 2):
                evaluation = 0
            else:
                evaluation, _ = minimax(position, depth-1, alpha, beta, False)
            position.pop()
            if evaluation > maxEval:
                maxEval = evaluation
                best_move = move
            alpha = max(alpha, evaluation)
            if beta <= alpha:
                break
        value = maxEval
    else:
        minEval = float('inf')
        for move in moves:
            position.push(move)
            if is_repetition_n_or_more(position, 2):
                evaluation = 0
            else:
                evaluation, _ = minimax(position, depth-1, alpha, beta, True)
            position.pop()
            if evaluation < minEval:
                minEval = evaluation
                best_move = move
            beta = min(beta, evaluation)
            if beta <= alpha:
                break
        value = minEval

    flag = 'exact'
    if value <= alpha_orig:
        flag = 'upperbound'
    elif value >= beta_orig:
        flag = 'lowerbound'
    if state_key is not None:
        transposition_table[state_key] = Entry(value, depth, flag, best_move)

    return value, best_move



In [4]:
TB_DIR = "../tablebases/gaviota"

# 26 ruchów do mata
# 3k1r2/2rnb3/8/8/3N4/1QB1N3/2K5/8 w - - 0 1

transposition_table.clear()
board = ZobristBoard("3k1r2/2rnb3/8/8/3N4/1QB1N3/2K5/8 w - - 0 1")

white = True
depth = 6
move_count = 0

start = time.time()
while not board.is_game_over():
    with chess.gaviota.open_tablebase(TB_DIR) as tb:
        wdl = tb.get_wdl(board)
        if wdl is not None:
            move_info, move = get_move_from_table(board, tb)
        else:
            score, move = minimax(board, depth, float('-inf'), float('inf'), white)
        
        if move is None:
            break

        board.push(move)
        move_count += 1

        white = not white

print("Minimax:", board.result(), "after", move_count, "moves,", "in", time.time() - start, "seconds.")

transposition_table.clear()
board = ZobristBoard("3k1r2/2rnb3/8/8/3N4/1QB1N3/2K5/8 w - - 0 1")

white = True
depth = 6
move_count = 0

start = time.time()
while not board.is_game_over():
    with chess.gaviota.open_tablebase(TB_DIR) as tb:
        wdl = tb.get_wdl(board)
        if wdl is not None:
            move_info, move = get_move_from_table(board, tb)
        else:
            score, move = minimax_with_qs(board, depth, float('-inf'), float('inf'), white)
        
        if move is None:
            break

        board.push(move)
        move_count += 1

        white = not white

print("Minimax with QS:", board.result(), "after", move_count, "moves,", "in", time.time() - start, "seconds.")

transposition_table.clear()
board = ZobristBoard("3k1r2/2rnb3/8/8/3N4/1QB1N3/2K5/8 w - - 0 1")

white = True
depth = 6
move_count = 0

start = time.time()
while not board.is_game_over():
    with chess.gaviota.open_tablebase(TB_DIR) as tb:
        wdl = tb.get_wdl(board)
        if wdl is not None:
            move_info, move = get_move_from_table(board, tb)
        else:
            score, move = pvs(board, depth, float('-inf'), float('inf'), white)
        
        if move is None:
            break

        board.push(move)
        move_count += 1

        white = not white

print("PVS:", board.result(), "after", move_count, "moves,", "in", time.time() - start, "seconds.")

transposition_table.clear()
board = ZobristBoard("3k1r2/2rnb3/8/8/3N4/1QB1N3/2K5/8 w - - 0 1")

white = True
depth = 6
move_count = 0

start = time.time()
while not board.is_game_over():
    with chess.gaviota.open_tablebase(TB_DIR) as tb:
        wdl = tb.get_wdl(board)
        if wdl is not None:
            move_info, move = get_move_from_table(board, tb)
        else:
            score, move = pvs_with_qs(board, depth, float('-inf'), float('inf'), white)
        
        if move is None:
            break

        board.push(move)
        move_count += 1

        white = not white

print("PVS with QS:", board.result(), "after", move_count, "moves,", "in", time.time() - start, "seconds.")

transposition_table.clear()
board = ZobristBoard("3k1r2/2rnb3/8/8/3N4/1QB1N3/2K5/8 w - - 0 1")

white = True
depth = 6
move_count = 0

start = time.time()
while not board.is_game_over():
    with chess.gaviota.open_tablebase(TB_DIR) as tb:
        wdl = tb.get_wdl(board)
        if wdl is not None:
            move_info, move = get_move_from_table(board, tb)
        else:
            best_value, move, reached, elapsed = iterative_deepening(board, depth, float('-inf'), float('inf'), white)
        
        if move is None:
            break

        board.push(move)
        move_count += 1

        white = not white

print("Iterative Deepenings:", board.result(), "after", move_count, "moves,", "in", time.time() - start, "seconds.")

transposition_table.clear()
board = ZobristBoard("3k1r2/2rnb3/8/8/3N4/1QB1N3/2K5/8 w - - 0 1")

white = True
depth = 6
move_count = 0

start = time.time()
while not board.is_game_over():
    with chess.gaviota.open_tablebase(TB_DIR) as tb:
        wdl = tb.get_wdl(board)
        if wdl is not None:
            move_info, move = get_move_from_table(board, tb)
        else:
            best_value, move, reached, elapsed = iterative_deepening_with_qs(board, depth, float('-inf'), float('inf'), white)
        
        if move is None:
            break

        board.push(move)
        move_count += 1

        white = not white

print("Iterative Deepening with QS:", board.result(), "after", move_count, "moves,", "in", time.time() - start, "seconds.")

Minimax: 1-0 after 79 moves, in 910.568306684494 seconds.
Minimax with QS: 1-0 after 29 moves, in 466.7320408821106 seconds.
PVS: 1-0 after 51 moves, in 324.9234895706177 seconds.
PVS with QS: 1-0 after 31 moves, in 3511.697776556015 seconds.
Iterative Deepenings: 1-0 after 15 moves, in 1278.6889297962189 seconds.
Iterative Deepening with QS: 1-0 after 25 moves, in 6688.772430419922 seconds.
